In [1]:
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt/code")' Datasets ModelSupport

Installing packages:
	.package(path: "/notebooks/language2motion.gt/code")
		Datasets
		ModelSupport
With SwiftPM flags: ['-c', 'release']
Working in: /tmp/tmp4sq0war8/swift-install
[1/2] Compiling Datasets ArrayUtils.swift
[2/3] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[3/3] Linking libjupyterInstalledPackages.so
Initializing Swift...
Installation complete!


In [2]:
import Foundation
import TensorFlow
import Datasets
import ModelSupport

In [3]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

('inline', 'module://ipykernel.pylab.backend_inline')


# load dataset

In [4]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")

In [5]:
let datasetFolderURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/2017-06-22/")

In [ ]:
public class MotionDataset: Codable {
    public let datasetFolderURL: URL
    public var motionSamples: [MotionSample]
    public var maxSampleID = 3966

    public init(datasetFolderURL: URL, grouppedJoints: Bool = true, normalized: Bool = true, sampled: Int? = nil) {
        self.datasetFolderURL = datasetFolderURL
        var motionSamples: [MotionSample] = []
        let fm = FileManager()
        
        var sampleIDs: [Int] = Array<Int>((0...maxSampleID))
        if sampled != nil {
            sampleIDs = Array(sampleIDs.choose(sampled!))
        }
        
        for sampleID in sampleIDs {
            let mmmFilename = String(format: "%05d_mmm.xml", sampleID)
            let annotationsFilename = String(format: "%05d_annotations.json", sampleID)
            print("Sample \(sampleID), \(mmmFilename), \(annotationsFilename)")
            
            let mmmURL = datasetFolderURL.appendingPathComponent(mmmFilename)
            let annotationsURL = datasetFolderURL.appendingPathComponent(annotationsFilename)
            
            if fm.fileExists(atPath: mmmURL.path) {
                let motionSample = MotionSample(sampleID: sampleID, mmmURL: mmmURL, annotationsURL: annotationsURL, grouppedJoints: grouppedJoints, normalized: normalized)            
                motionSamples.append(motionSample)
            } else {
                print("** Sample \(sampleID) doesn't exist.")
            }
        }
        self.motionSamples = motionSamples
    }

    // TODO: code throwing errors
    public init(from serializedDatasetURL: URL) {
        let data: Data = FileManager.default.contents(atPath: serializedDatasetURL.path)!
        let motionData = try! PropertyListDecoder().decode(MotionDataset.self, from: data)
        datasetFolderURL = motionData.datasetFolderURL
        motionSamples = motionData.motionSamples
        maxSampleID = motionData.maxSampleID
    }

    // TODO: code throwing errors
    public func write(to serializedDatasetURL: URL) {
        let encoder = PropertyListEncoder()
        encoder.outputFormat = .binary
        let mdData = try! encoder.encode(self)
        try! mdData.write(to: serializedDatasetURL) 
    }

    public var description: String {
        return "MotionDataset(motionSamples: \(motionSamples.count))"
    }
}

In [ ]:
let motionDataset = MotionDataset(datasetFolderURL: datasetFolderURL, grouppedJoints: false, normalized: true, sampled: 2)

# downsample motion

In [6]:
let grouppedJoints: Bool = false
let normalized: Bool = true
let sampled: Int? = 2

In [7]:
let fm = FileManager()

var motionSamples: [MotionSample] = []

In [8]:
let sampleID = 1

In [9]:
let mmmFilename = String(format: "%05d_mmm.xml", sampleID)
let annotationsFilename = String(format: "%05d_annotations.json", sampleID)
print("Sample \(sampleID), \(mmmFilename), \(annotationsFilename)")

let mmmURL = datasetFolderURL.appendingPathComponent(mmmFilename)
let annotationsURL = datasetFolderURL.appendingPathComponent(annotationsFilename)

Sample 1, 00001_mmm.xml, 00001_annotations.json


In [10]:

let mmm_doc = MotionSample.loadMMM(fileURL: mmmURL)
let jointNames = MotionSample.getJointNames(mmm_doc: mmm_doc)

let motionFrames = MotionSample.getMotionFrames(mmm_doc: mmm_doc, jointNames: jointNames)

let annotations = MotionSample.getAnnotations(fileURL: annotationsURL)
let timestamps: [Float] = motionFrames.map { $0.timestamp }
let timestampsArray = ShapedArray<Float>(shape: [timestamps.count], scalars: timestamps)
let motionFramesArray = MotionSample.getJointPositions(motionFrames: motionFrames, grouppedJoints: grouppedJoints, normalized: normalized)

In [36]:
var motionFramesBuckets = [[MotionFrame]](repeating: [], count:10)
var timestampsBuckets = [[Float]](repeating: [], count:10)

In [38]:
for idx in 0..<motionFrames.count {
    let bucket = idx % 10
    //let b_idx = idx / 10
    //print(idx, bucket, b_idx)
    motionFramesBuckets[bucket].append(motionFrames[idx])
    timestampsBuckets[bucket].append(timestamps[idx])
}
for (idx, mfb) in motionFramesBuckets.enumerated() {
    print(idx, mfb.count)
}

0 38
1 38
2 38
3 38
4 38
5 38
6 38
7 38
8 37
9 37


In [13]:
timestampsArray.shape

▿ 1 element
  - 0 : 378


In [11]:
motionFramesArray.shape

▿ 2 elements
  - 0 : 378
  - 1 : 45


In [ ]:
if fm.fileExists(atPath: mmmURL.path) {
    let motionSample = MotionSample(sampleID: sampleID, mmmURL: mmmURL, annotationsURL: annotationsURL, grouppedJoints: grouppedJoints, normalized: normalized)            
    motionSamples.append(motionSample)
} else {
    print("** Sample \(sampleID) doesn't exist.")
}